In [1]:
from binance.client import Client
import pandas as pd
from binance.enums import *

import random

In [2]:
class Config:
    def __init__(self, api_key: str, api_secret: str):
        self.__api_key = api_key
        self.__api_secret = api_secret                

    def get_api_key(self):
        return self.__api_key

    def get_api_secret(self):
        return self.__api_secret

class BinanceClient():

    def __init__(self, config: Config, quote_asset: str, testnet=False):
        self.__api_key = config.get_api_key()
        self.__api_secret = config.get_api_secret()
        self.__tld = "com"
        self.__testnet = testnet
        self.client = Client(api_key=self.__api_key, api_secret=self.__api_secret,
                             tld=self.__tld, testnet=self.__testnet) 
        
        self.__quote_asset = quote_asset          

    def get_klines(self, symbol: str, interval, limit=1000):
            try:
                klines = self.client.get_klines(
                    symbol=symbol, interval=interval, limit=limit)
                return klines
            except Exception as e:
                print(
                    "An exception occured: {0} at BinanceClient/get_klines()".format(e))

    def get_klines_properly(self, symbol: str, interval, limit=1000):
            try:
                klines = self.get_klines(
                    symbol=symbol, interval=interval, limit=limit)
                
                times = pd.Series([float(x[0]) for x in klines])
                opens = pd.Series([float(x[1]) for x in klines])
                highs = pd.Series([float(x[2]) for x in klines])
                lows = pd.Series([float(x[3]) for x in klines])
                closes = pd.Series([float(x[4]) for x in klines])
                volumes = pd.Series([float(x[5]) for x in klines])

                return times, opens, highs, lows, closes, volumes

            except Exception as e:
                print(
                    "An exception occured: {0} at BinanceClient/get_klines_properly()".format(e))
                
    def get_exchange_info(self):
        try:
            exchange_info = self.client.get_exchange_info()
            return exchange_info
        except Exception as e:
            print(
                "An exception occured: {0} at BinanceClient/get_exchange_info()".format(e))


    def get_exchange_info_symbols(self):
        try:
            symbols = self.get_exchange_info()["symbols"]
            return symbols
        except Exception as e:
            print(
                "An exception occured: {0} at BinanceClient/get_exchange_info_symbols()".format(e))

    def get_quote_asset(self):
        return self.__quote_asset
    
class Coin:
    __symbol_list = []

    @classmethod
    def arange_symbol_list(cls, client: BinanceClient):
        try:
            available_symbols_list = list(cls.eliminate_symbols(
                cls, client, symbols_dict=cls.load_symbols(client)).keys())

            return available_symbols_list                      
        except Exception as e:
            print(
                "An exception occured: {0} at Coin/arange_symbol_list()".format(e))

    @staticmethod
    def eliminate_symbols(cls, client: BinanceClient, symbols_dict: dict):
        try:
            available_symbols_dict = {outer_k:
                                      {inner_k: inner_v for inner_k,
                                       inner_v in outer_v.items()}
                                      for outer_k, outer_v in symbols_dict.items()
                                      if outer_v["quoteAsset"] == client.get_quote_asset() and outer_v["isSpotTradingAllowed"] == True}

            return available_symbols_dict
        except Exception as e:
            print(
                "An exception occured: {0} at Coin/eliminate_symbols()".format(e))

    @classmethod
    def load_symbols(cls, client: BinanceClient):
        try:
            symbols_dict = dict()
            symbols = client.get_exchange_info_symbols()

            for curr_symbol in symbols:
                symbol_name = curr_symbol["symbol"]
                symbol_base_asset = curr_symbol["baseAsset"]
                symbol_quote_asset = curr_symbol["quoteAsset"]
                symbol_is_spot_trading_allowed = curr_symbol["isSpotTradingAllowed"]

                symbols_dict[symbol_name] = {"baseAsset": symbol_base_asset,
                                             "quoteAsset": symbol_quote_asset,
                                             "isSpotTradingAllowed": symbol_is_spot_trading_allowed}
            return symbols_dict
        except Exception as e:
            print("An exception occured: {0} at Coin/load_symbols()".format(e))

In [3]:
config = Config(api_key="",
                    api_secret="")
main_client = BinanceClient(config=config, quote_asset = "USDT")

In [4]:
def pick_symbols(main_client: BinanceClient, interval_short, interval_long):
    upwards_list, downwards_list, sideways_list = eliminate_symbols(classify_symbols(get_available_symbols(main_client), interval_short), interval_short)
    random_upwards, random_downwards, random_sideways = pick_random_symbols(upwards_list, downwards_list, sideways_list)    
    print(random_upwards, random_downwards, random_sideways)
    
    save_to_csv(random_upwards, random_downwards, random_sideways, interval_short)
    save_to_csv(random_upwards, random_downwards, random_sideways, interval_long)    

def get_available_symbols(main_client: BinanceClient):
    return Coin.arange_symbol_list(main_client)

def classify_symbols(available_symbols_list: list, interval):
    market_side_dict = {}
    for curr_symbol in available_symbols_list:
        closes = get_closes(interval, curr_symbol)
        if (100 * (closes[-1] - closes[0])) / closes[0] >= 10:
            market_side_dict[str(curr_symbol)] = "upwards"
        elif (100 * (closes[-1] - closes[0])) / closes[0] <= -10:
            market_side_dict[str(curr_symbol)] = "downwards"
        elif (100 * (closes[-1] - closes[0])) / closes[0] <= 10 and (100 * (closes[-1] - closes[0])) / closes[0] >= -10:
            market_side_dict[str(curr_symbol)] = "sideways"       
    
    return market_side_dict
    
def eliminate_symbols(market_side_dict: dict, interval):    
    upwards_list = [key for key, value in market_side_dict.items() if value == "upwards"]
    downwards_list = [key for key, value in market_side_dict.items() if value == "downwards"]
    sideways_list = [key for key, value in market_side_dict.items() if value == "sideways"]
    
    return upwards_list, downwards_list, sideways_list

def pick_random_symbols(upwards_list: list, downwards_list: list, sideways_list: list):
    random_index = random.sample(range(0, len(upwards_list)), 5 if len(upwards_list) >= 5 else len(upwards_list))
    random_upwards = [upwards_list[index] for index in random_index]    
    
    random_index = random.sample(range(0, len(downwards_list)), 5 if len(downwards_list) >= 5 else len(downwards_list))
    random_downwards = [downwards_list[index] for index in random_index]
    
    random_index = random.sample(range(0, len(sideways_list)), 5 if len(sideways_list) >= 5 else len(sideways_list))
    random_sideways = [sideways_list[index] for index in random_index]
    
    return random_upwards, random_downwards, random_sideways

def save_to_csv(random_upwards: list, random_downwards: list, random_sideways: list, interval):
    target_symbol_list = [*random_upwards, *random_downwards, *random_sideways]
    for curr_symbol in target_symbol_list:        
        curr_result = main_client.get_klines_properly(str(curr_symbol), interval = interval)
        curr_df = pd.DataFrame()    
        curr_df["times"], curr_df["opens"], curr_df["highs"], curr_df["lows"], curr_df["closes"], curr_df["volumes"] = curr_result
        if interval == KLINE_INTERVAL_1HOUR:
            curr_df.to_csv("1h_{}.csv".format(str(curr_symbol)))
        elif interval == KLINE_INTERVAL_15MINUTE:
            curr_df.to_csv("15m_{}.csv".format(str(curr_symbol)))   
            
def get_closes(interval, curr_symbol):        
    curr_result = main_client.get_klines_properly(str(curr_symbol), interval = interval)
    curr_df = pd.DataFrame()    
    curr_df["times"], curr_df["opens"], curr_df["highs"], curr_df["lows"], curr_df["closes"], curr_df["volumes"] = curr_result
    return list(curr_df["closes"])     
    

In [ ]:
pick_symbols(main_client, KLINE_INTERVAL_15MINUTE, KLINE_INTERVAL_1HOUR)